In [42]:
import numpy as np
import cv2
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers import LeakyReLU
import keras
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [20]:
data = np.load('Dogs_npz.npz')
print(data.files)

['arr_0', 'arr_1', 'arr_2', 'arr_3']


In [21]:
X_train = data['arr_0']
X_test = data['arr_2']
Y_train = data['arr_1']
Y_test = data['arr_3']

In [22]:
print(X_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(Y_train.shape)
X_train = X_train/255
X_test = X_test/255

(8646, 90, 90, 3)
(700, 90, 90, 3)
(700, 70)
(8646, 70)


In [23]:
input_shape = X_test.shape[1]

In [24]:
# improve this
# activation for hidden layers is not sigmoid and relu because of vanishing gradient problem
# relu is the best one for cnns (seems to be standard practice from readings)
# for the output layer I chose softmax because it produces a vector of values that sums up to 1.
# Where the values indicate the probability of the image belonging to a particular class. 
# This is the correct one to use for multiclass classification.
# relu and sigmoid have the problem of causing dead neurons (0 values for the weights). 
# so we are using leakyrelu so that "dead neurons" will be replaced with very low values.
# use activation linear in hidden layers and then leakyrelu as a separate layer
model=Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', input_shape=(input_shape, input_shape, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))
model.add(Dense(70, activation='softmax'))

In [25]:
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 90, 90, 16)        208       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 90, 90, 16)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 45, 45, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 45, 45, 32)        2080      
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 45, 45, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 22, 22, 64)       

In [26]:
history = model.fit(X_train, Y_train, epochs=40, validation_split=0.08096)

Train on 7946 samples, validate on 700 samples
Epoch 1/40
7946/7946 [==============================] - 5s 647us/step - loss: 4.1641 - accuracy: 0.0254 - val_loss: 4.0629 - val_accuracy: 0.0329
Epoch 2/40
7946/7946 [==============================] - 5s 579us/step - loss: 3.9289 - accuracy: 0.0544 - val_loss: 3.7120 - val_accuracy: 0.0814
Epoch 3/40
7946/7946 [==============================] - 4s 561us/step - loss: 3.7438 - accuracy: 0.0753 - val_loss: 3.6210 - val_accuracy: 0.0743
Epoch 4/40
7946/7946 [==============================] - 4s 560us/step - loss: 3.6389 - accuracy: 0.0982 - val_loss: 3.4994 - val_accuracy: 0.0914
Epoch 5/40
7946/7946 [==============================] - 4s 557us/step - loss: 3.5487 - accuracy: 0.1157 - val_loss: 3.4087 - val_accuracy: 0.1414
Epoch 6/40
7946/7946 [==============================] - 5s 601us/step - loss: 3.4071 - accuracy: 0.1450 - val_loss: 3.2212 - val_accuracy: 0.1543
Epoch 7/40
7946/7946 [==============================] - 5s 624us/step - loss:

In [36]:
Y_hat = model.predict(X_train)

In [37]:
Y_hat_test = model.predict(X_test)

In [38]:
cm = confusion_matrix(Y_test.argmax(axis=1), Y_hat_test.argmax(axis=1))

In [48]:
cr = classification_report(Y_test.argmax(axis=1), Y_hat_test.argmax(axis=1), output_dict=True)
# it gives you a warning because some labels are never predicted

C:\Users\prath\anaconda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prath\anaconda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prath\anaconda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
# print(cm)
# print(cr)

In [52]:
df = pd.DataFrame(cr).transpose()
df.to_csv(r'C:\Users\prath\Downloads\Machine_Learning\Dogs\classification_report.csv')

In [53]:
# # transfer learning using vgg16 and resnet50
# # i think we should try other models and see which ones provide the best results
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16

In [54]:
# base_model = tf.keras.applications.VGG16(input_shape=((224,224,3)),include_top=False)
# base_model.summary()

In [55]:
# image_batch, label_batch = next(iter(np.load(X_train)))
# feature_batch = base_model(image_batch)
# print(feature_batch.shape)
# # model_vgg16.add()
# # model_vgg16.add(GlobalAveragePooling2D(input_shape=(224,224,3)))
# # model_vgg16.add(Dense(70, activation='softmax'))
# # model_vgg16.summary()
# model_vgg16.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])